In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [2]:
device = 'cuda'

In [3]:
mnist_train = dsets.MNIST(root ='MNIST_data/',
                         train = True,
                         transform=transforms.ToTensor(),
                         download=True)
mnist_test = dsets.MNIST(root='MNIST_data/',
                        train=False,
                        transform=transforms.ToTensor(),
                        download=True)

In [4]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                         batch_size = 100,
                                         shuffle = True,
                                         drop_last=True)

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32,64,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        
        # L3 ImgIn shape=(?, 7, 7, 64)
        #    Conv      ->(?, 7, 7, 128)
        #    Pool      ->(?, 4, 4, 128)
        self.layer3 = nn.Sequential(
            nn.Conv2d(64,128,3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        )
        
        # L4 FC 4x4x128 inputs -> 625 outputs
        self.fc1 = nn.Linear(4*4*128,625, bias = True)
        nn.init.xavier_uniform_(self.fc1.weight)
        self.layer4 = nn.Sequential(
            self.fc1,
            nn.ReLU(),
            nn.Dropout(p= 0.5)
        )
        
        # L5 Final FC 625 inputs -> 10 outputs
        self.fc2 = nn.Linear(625,10,bias=True)
        nn.init.xavier_uniform_(self.fc2.weight)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.view(out.size(0),-1)
        out = self.layer4(out)
        out = self.fc2(out)
        return out

In [6]:
model = CNN().to(device)

In [7]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)

In [8]:
total_batch = len(data_loader)
print(total_batch)

600


In [9]:
for epoch in range(15):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        
        cost.backward()
        optimizer.step()
        
        avg_cost += cost / total_batch
        
    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

[Epoch:    1] cost = 0.210533321
[Epoch:    2] cost = 0.0527158231
[Epoch:    3] cost = 0.0402955599
[Epoch:    4] cost = 0.0302373637
[Epoch:    5] cost = 0.0245347377
[Epoch:    6] cost = 0.0198029447
[Epoch:    7] cost = 0.017360149
[Epoch:    8] cost = 0.0162010323
[Epoch:    9] cost = 0.0141512919
[Epoch:   10] cost = 0.0116930772
[Epoch:   11] cost = 0.00964834727
[Epoch:   12] cost = 0.0110444063
[Epoch:   13] cost = 0.0081012696
[Epoch:   14] cost = 0.00839664601
[Epoch:   15] cost = 0.00761942752


In [ ]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test),1,28,28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)
    
    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction,1) == Y_test
    accuracy = correct_prediction.float()